This notebook is used for development of helper functions to evantually be included in the feature explorer module.

In [2]:
import pandas as pd

## Load Datasets for testing

In [3]:
# King County
king = pd.read_csv('https://raw.githubusercontent.com/ryanleeallred/datasets/master/kc_house_data.csv')

# Water Pumps
pumps = pd.read_csv('Data/pumps.csv')
pumps_target = pd.read_csv('Data/pump_train_labels.csv')
pumps_target = pumps_target['status_group']

# Moores Law
tables = pd.read_html('https://en.wikipedia.org/wiki/Transistor_count', header=0)
moore = tables[0]

# Pedestrian Crossing
crossing = pd.read_csv('Data/daily.csv')

king.shape, pumps.shape, moore.shape, crossing.shape

((21613, 21), (59400, 40), (139, 6), (1063, 9))

In [4]:
king.head()

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,7129300520,20141013T000000,221900.0,3,1.00,1180,5650,1.0,0,0,...,7,1180,0,1955,0,98178,47.5112,-122.257,1340,5650
1,6414100192,20141209T000000,538000.0,3,2.25,2570,7242,2.0,0,0,...,7,2170,400,1951,1991,98125,47.7210,-122.319,1690,7639
2,5631500400,20150225T000000,180000.0,2,1.00,770,10000,1.0,0,0,...,6,770,0,1933,0,98028,47.7379,-122.233,2720,8062
3,2487200875,20141209T000000,604000.0,4,3.00,1960,5000,1.0,0,0,...,7,1050,910,1965,0,98136,47.5208,-122.393,1360,5000
4,1954400510,20150218T000000,510000.0,3,2.00,1680,8080,1.0,0,0,...,8,1680,0,1987,0,98074,47.6168,-122.045,1800,7503


### Cardinality per Feature Function

In [5]:
def get_cardinality(df, include_numeric=True):
    '''
    Returns: Pandas Series, containing each of the column names as the index values and 
    a count of the unique values within each column as the data.
    
    Parameter: Pandas DataFrame, include_numeric -- should numeric columns be included
    '''
    if include_numeric: 
        cols = df.columns
    else: 
        cols = df.select_dtypes(exclude='number').columns
    uniques = []
    for col in cols:
        uniques.append(df[col].nunique())
    uniques = pd.Series(uniques, cols)
    return uniques

get_cardinality(pumps)[:5]

id               59400
amount_tsh          98
date_recorded      356
funder            1897
gps_height        2428
dtype: int64

### Detect High Cardinality Features Function

In [6]:
def get_high_cardinality(df, threshold=20): 
    ''' 
    Returns a Pandas Series containing a list of the non-numeric columns which contain more than 20 unique values
    
    Parameters: Pandas Dataframe, threshold to determine cutoff point for high cardinality 
    '''
    cardinality = get_cardinality(df, include_numeric=False)
    high_card = cardinality >= threshold
    return cardinality[high_card]
    
get_high_cardinality(pumps, 20)

date_recorded      356
funder            1897
installer         2145
wpt_name         37400
subvillage       19287
region              21
lga                125
ward              2092
scheme_name       2696
dtype: int64

### Get Sorted List of Correlations with Specificed Target

In [7]:
def corr_with_target(features, target):
    '''
    Returns a Pandas Series containing the correlations of each feature with the specific target.
    Correlation values will not be returned for non-numeric datatypes. The length of the target Series must 
    equal the dataframe.
    
    If target is non-numeric, it will be ordinal encoded. This may imply an ordering to the target classes that is improper. 
    '''
    if ( target.dtype != ('int' or 'float') ):
        #TODO ordinal encode target
        target = pd.Series( target.factorize()[0] )
        target = target.rename('target')
    # Concat features and target   
    df = pd.concat([features, target], axis=1)
    corrs = df.corr()
    
    # Drop target's correlation with itself
    corrs_target = corrs['target'].sort_values(ascending=False)[1:]
    
    return corrs_target

corr_with_target(king, king['price'])

price            0.213099
sqft_above       0.175123
grade            0.172603
sqft_living      0.168968
sqft_living15    0.156909
bathrooms        0.146990
floors           0.110575
yr_built         0.100722
bedrooms         0.074662
view             0.061059
waterfront       0.036614
long             0.032638
sqft_basement    0.022977
yr_renovated     0.017716
lat              0.003929
id              -0.001906
sqft_lot        -0.009207
sqft_lot15      -0.017667
condition       -0.025234
zipcode         -0.033989
Name: target, dtype: float64

### Detect Suspicious Values

In [8]:
def detect_suspicious(df, custom_vals=[]):
    '''
    Provides a count of suspicious values and returns a dataframe with the counts and the suspicious values.
    In addition, to checking for common suspicious values (e.g., 'N/A') a list of custom suspicious values 
    can be passed.
    '''
    suspicious_vals = ['Not Known', 'Unknown', 'None', 'Not known', 'not known', '-', 
                       'unknown', '##', 'none', '?', '??', '0', -9, -99, 
                       -999, -9999, 9999, 66, 77, 88, "NA", "N A", "N/A", "NA ", " NA", "N /A", 
                       "N / A", " N / A", "N / A ", "na", "n a", "n/a", "na ", " na", "n /a", 
                       "n / a", " a / a", "n / a ", "NULL", "null", "", "\\?", "\\*", "\\."]
    suspicious_vals = suspicious_vals + custom_vals
    cols = df.columns
    suspect_cols = []
    suspect_counts = []
    suspect_vals = []
    for col in cols:
        suspect = (df[col].isin(suspicious_vals)).sum()
        if (suspect > 0):
            suspect_cols.append(col)
            suspect_counts.append(suspect)
            
            # Find intersection of values in suspicious_vals and the column
            vals = list( set(df[col]) & set(suspicious_vals) )
            suspect_vals.append(vals)
    frame = pd.DataFrame({
        'Suspicious Count': suspect_counts,
        'Suspect Values': suspect_vals
    }, index=suspect_cols)
    return frame

detect_suspicious(pumps)

,Suspicious Count,Suspect Values
id,4,"[88, 66, 77, 9999]"
funder,810,"[None, 0, Not Known, Unknown]"
gps_height,97,"[88, 66, 77, -9]"
installer,825,"[-, unknown, Unknown, Not known, 0, not known]"
wpt_name,3588,"[None, Not Known, none, Unknown]"
subvillage,1,[##]
population,20,"[88, 66, 77]"
scheme_management,1,[None]
scheme_name,677,"[None, none, not known]"
management,561,[unknown]


## Outlier Detection 
- Methods adapted from: http://colingorrie.github.io/outlier-detection.html#fn:2

In [9]:
def detect_outliers(df, method='modified_z_score', threshold=3.5):
    if method == 'standard_z_score':
        result = outlier_standard_z_score(df, threshold)
    elif method == 'outlier_iqr':
        result = outlier_iqr(df, threshold)
    else:
        result = outlier_modified_z_score(df, threshold)
    return result

detect_outliers(pumps, threshold=3.5)

NameError: name 'outlier_modified_z_score' is not defined

### Modified Z Score Outlier Detection 

In [ ]:
def outlier_modified_z_score(df, threshold):
    num_cols = df.select_dtypes(include='number')
    outlier_cols = []
    outlier_counts = []
    outlier_vals = []
    for col in num_cols.columns:
        outliers = []
        median = np.median(df[col])
        median_absolute_deviation = np.median([np.abs(i - median) for i in df[col]])

        modified_z_scores = []
        for i in df[col]:
            if median_absolute_deviation != 0:
                modified_z_scores.append( 0.675 * (i-median) / median_absolute_deviation )
            else: 
                modified_z_scores.append( 0 )


        outliers = np.where(np.abs(modified_z_scores) > z_score_threshold)

        outliers_set = set()
        for i in outliers[0]:
            outlier_val = df[col].iloc[i]
            outliers_set.add(outlier_val)

        if outliers[0].size > 0:
            outlier_cols.append(col)
            outlier_counts.append(len(outliers[0]))
            outlier_vals.append(outliers_set)

    frame = pd.DataFrame({
            'Outlier Count': outlier_counts,
            'Outlier Values': outlier_vals
    }, index=outlier_cols)

    return frame

outlier_modified_z_score(pumps, 5)

### Z Score Outlier Detection

In [ ]:
def outlier_z_score(df, threshold):
    num_cols = df.select_dtypes(include='number')
    outlier_cols = []
    outlier_counts = []
    outlier_vals = []
    for col in num_cols.columns:
        outliers = []
        mean = np.mean(df[col])
        stdev = np.std(df[col])
        z_scores = [(i - mean) / stdev for i in df[col]]

        outliers = np.where(np.abs(z_scores) > threshold)
        
        outliers_set = set()
        for i in outliers[0]:
            outlier_val = df[col].iloc[i]
            outliers_set.add(outlier_val)


        if outliers[0].size > 0:
            outlier_cols.append(col)
            outlier_counts.append(len(outliers[0]))
            outlier_vals.append(outliers_set)

    frame = pd.DataFrame({
            'Outlier Count': outlier_counts,
            'Outlier Values': outlier_vals
        }, index=outlier_cols)

    return frame

outlier_z_score(pumps, 5)

### IQR Outlier Detection 

In [ ]:
def outlier_iqr(df, threshold):
    num_cols = df.select_dtypes(include='number')
    outlier_cols = []
    outlier_counts = []
    outlier_vals = []
    for col in num_cols.columns:
        outliers = []
        quartile_1, quartile_3 = np.percentile(df[col], [25, 75])
        iqr = quartile_3 - quartile_1
        lower = quartile_1 - (iqr * threshold)
        upper = quartile_3 + (iqr * threshold)

        outliers = np.where( (df[col] > upper) | (df[col] < lower) )
        
        outliers_set = set()
        for i in outliers[0]:
            outlier_val = df[col].iloc[i]
            outliers_set.add(outlier_val)

        if outliers[0].size > 0:
            outlier_cols.append(col)
            outlier_counts.append(len(outliers[0]))
            outlier_vals.append(outliers_set)

    frame = pd.DataFrame({
            'Outlier Count': outlier_counts,
            'Outlier Values': outlier_vals
        }, index=outlier_cols)

    return frame

outlier_iqr(pumps, 5)

# Data Dictionary
- Output a detailed description of a DataFrame which can be used to inform a data scientist's cleaning, imputation, and feature engineering process

In [ ]:
def data_dict(df, target):

    null = df.isnull().sum()
    null = pd.DataFrame(null)
    null.columns = ['Null Values']

    unique = df.nunique()
    unique = pd.DataFrame(unique)
    unique.columns = ['Unique Values']

    cardinality = get_cardinality(df)
    cardinality = pd.DataFrame(cardinality)
    cardinality.columns = ['Cardinality']

    datatype = df.dtypes
    datatype = pd.DataFrame(datatype)
    datatype.columns = ['Datatype']

    skew = df.skew()
    skew = pd.DataFrame(skew)
    skew.columns = ['Skew']

    corr_w_target = corr_with_target(df, target)
    corr_w_target = pd.DataFrame(corr_w_target)
    corr_w_target.columns = ['Correlation W/Target']

    suspicious = detect_suspicious(df)
    outlier = detect_outliers(df)
    
    frames = [null, cardinality, datatype, skew, suspicious, corr_w_target, outlier]

    combined = unique
    for frame in frames:
        combined = pd.merge(combined, frame, how='outer', left_index=True, right_index=True)

    counts = ['Suspicious Count', 'Outlier Count']
    combined[counts] = combined[counts].fillna(0)
    vals = ['Suspect Values', 'Outlier Values']
    combined[vals] = combined[vals].fillna('None')
    non_num = ['Skew', 'Correlation W/Target']
    combined[non_num] = combined[non_num].fillna('Non-Numeric')

    col_ordered = ['Datatype', 'Unique Values', 'Null Values', 'Cardinality', 'Correlation W/Target', 'Skew', 'Outlier Count', 'Outlier Values', 'Suspicious Count', 'Suspect Values']
    combined = combined[col_ordered]
    
    return combined

In [ ]:
data_dict(pumps, target)

In [ ]:
data_dict(king, king['price'])

In [ ]:
data_dict(crossing, crossing['Total'])

In [ ]:
data_dict(moore, moore['Transistor count'])